### Check if you are able to get response from Claude through AWS Bedrock

In [14]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_claude",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

Hi!


In [15]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_titan",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Hello!


### Check if the API docs are fetched correctly

In [ ]:
import requests
weather_response = requests.get("http://localhost:8000/weather")

print(weather_response.json()["response"])

### Check if the user query is fetched correctly

In [2]:
import requests
weather_response = requests.post("http://localhost:8000/user_query", json={"user_query": "What is the weather in London?"})

print(weather_response.json())

{'response': 'What is the weather in London?'}


### Test the query with Claude as both the mother as well as the child agent

In [43]:
query_response = requests.post("http://localhost:8000/user_query", 
                             json={"user_query": "What is the weather like at latitude 37.7749 and longitude -122.4194"})
print(query_response.json())

agent_response = requests.get("http://localhost:8000/claude_child_agent", 
                            params={"user_query": query_response.json()["response"]})

{'response': 'What is the weather like at latitude 37.7749 and longitude -122.4194'}


### Test the query with Claude as the mother agent and Titan as the child agent

In [46]:
query_response = requests.post("http://localhost:8000/user_query", 
                             json={"user_query": "How is the humidity in Saulo Paulo Like?"})
print("Query Response:", query_response.json())

agent_response = requests.get("http://localhost:8000/titan_child_agent", 
                            params={"user_query": query_response.json()["response"]})
print("Agent Response:", agent_response.json())

Query Response: {'response': 'How is the humidity in Saulo Paulo Like?'}
Agent Response: {'message': 'Child agent successfully created', 'response': 'import boto3\nimport requests\nimport json\nimport os\nfrom bedrock_handler import call_titan\nfrom dotenv import load_dotenv\n\nload_dotenv()\n\napi_key = os.getenv("OPENWEATHERMAP_API_KEY")\n\ndef get_data(city):\n    api_endpoint = "http://api.openweathermap.org/data/2.5/weather"\n    params = {\n        "q": city,\n        "appid": api_key,\n        "units": "metric"\n    }\n    try:\n        response = requests.get(api_endpoint, params=params)\n        response.raise_for_status()\n        data = response.json()\n        temperature = data["main"]["temp"]\n        description = data["weather"][0]["description"]\n        weather_info = f"Temperature: {temperature} C, Weather: {description}"\n        return weather_info\n    except requests.exceptions.RequestException as e:\n        return f"Error: {e}"\n\nprompt = f"How is the humidity

In [32]:
import child_agent
!python child_agent.py

, Wind: 1.5 m/s, Humidity: 71%, Pressure: 1011 mb, Sunrise: 6:30 AM, Sunset: 5:30 PM
Based on the provided content, the weather in Delhi is currently 18 degrees Celsius with broken clouds, a wind speed of 1.5 m/s, a humidity of 71%, an atmospheric pressure of 1011 mb, and a sunrise time of 6:30 AM and a sunset time of 5:30 PM.


### Added a route to test and verify the health of the child agent code and execute it to get the response from the child agent

In [47]:
import requests

# The path to the agent file you want to verify
agent_file = "child_agent.py"

# Make the POST request
response = requests.post(f"http://localhost:8000/verify_agent?agent_file={agent_file}")

# Print the response
print(response.json())

{'status': 'success', 'syntax_valid': True, 'message': 'Agent file executed successfully', 'output': '\nThe humidity in Sao Paulo is high, with an average of 75% throughout the year. The city experiences a tropical climate with two distinct seasons: a wet season from November to March and a dry season from April to September. The humidity is highest during the wet season, with average humidity levels ranging from 70% to 80%. The dry season is slightly less humid, with average humidity levels ranging from 60% to 70%.\n\nThe humidity in Sao Paulo can have a significant impact on daily life. It can make it difficult to stay cool, as the air is often thick and sticky. It can also lead to the growth of mold and mildew, which can cause health problems. To combat the humidity, many people in Sao Paulo use air conditioning. Air conditioning is widely available in homes, offices, and public buildings, and it can help to keep the air cool and dry.\n\nHowever, air conditioning can also be expensi